In [ ]:
!pip install pandas selenium openpyxl pyperclip

In [ ]:
import time
import pandas as pd
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# === CONFIG ===
excel_file = "contacts.xlsx"   # Excel file with Phone, Name columns
delay_between = 8              # Seconds between messages

# === LOAD CONTACTS ===
df = pd.read_excel(excel_file)
contacts = df[['Phone', 'Name']].dropna()

# === OPEN WHATSAPP WEB ===
driver = webdriver.Chrome()
driver.get("https://web.whatsapp.com/")
print(">>> Scan the QR code in WhatsApp Web and press ENTER here...")
input()

# === SEND MESSAGES ===
for _, row in contacts.iterrows():
    number = str(row['Phone']).strip().replace(" ", "")
    if not number.startswith("91"):   # Ensure +91 prefix
        number = "91" + number

    name = str(row['Name']).strip()

    message_text = f"""Hello {name},

message 
""".strip()

    url = f"https://web.whatsapp.com/send?phone={number}&forceNewChat=true"
    driver.get(url)

    try:
        # ✅ Wait for actual chat input box (not search bar!)
        try:
            input_box = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located(
                    (By.XPATH, '//div[@contenteditable="true"][@aria-placeholder="Type a message"]')
                )
            )
        except TimeoutException:
            # Check if number is invalid
            try:
                driver.find_element(By.XPATH, '//div[contains(text(),"phone number shared via url is invalid")]')
                print(f"⚠️ Skipped {name} ({number}) - Not on WhatsApp")
                continue
            except NoSuchElementException:
                print(f"❌ Timeout waiting for chat {name} ({number})")
                continue

        # ✅ Paste message using clipboard (handles emojis, Unicode)
        pyperclip.copy(message_text)
        ActionChains(driver).move_to_element(input_box).click().perform()
        input_box.send_keys(Keys.CONTROL, "v")
        time.sleep(1)

        # ✅ Send message
        input_box.send_keys(Keys.ENTER)
        print(f"✅ Message sent to {name} ({number})")

    except Exception as e:
        print(f"❌ Failed for {name} ({number}): {e}")

    time.sleep(delay_between)

driver.quit()
